In [1]:
import numpy as np

## Encoder: VGG16, Decoder: 

### Obtain Bottleneck Features

In [2]:
bottleneck_features = np.load('bottleneck_features/Flicker8k_bottleneck_features_VGG16_avgpooling.npz')
bottleneck_features_train = bottleneck_features["train"]
bottleneck_features_test = bottleneck_features["test"]
bottleneck_features_validation = bottleneck_features["validation"]

### Model Architecture

In [3]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense, RepeatVector
from keras.layers import Input, Embedding
from keras.models import Sequential

/home/ubuntu/anaconda3/envs/tensorflow-3.5/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
bottleneck_features_train.shape[1]

512

In [11]:
vocab_size = 4408
embedding_size = 300
sentence_input = Input(shape=[None])
word_embedding = Embedding(input_dim=vocab_size, output_dim=embedding_size)(sentence_input)

In [ ]:
from keras.layers import BatchNormalization, Concatenate, LSTM, TimeDistributed
from keras.models import Model
from keras.optimizers import Adam

sequence_input = Concatenate(axis=1)([image_embedding, word_embedding])

learning_rate = 0.00051
lstm_output_size = 300
vocab_size = 2536
lstm_layers = 3
dropout_rate = 0.22
input_ = sequence_input

for _ in range(lstm_layers):
  input_ = BatchNormalization(axis=-1)(input_)
  lstm_out = LSTM(units=lstm_output_size,
                  return_sequences=True,
                  dropout=dropout_rate,
                  recurrent_dropout=dropout_rate)(input_)
  input_ = lstm_out
sequence_output = TimeDistributed(Dense(units=vocab_size))(lstm_out)

model = Model(inputs=[image_input, sentence_input],
              outputs=sequence_output)
model.compile(optimizer=Adam(lr=learning_rate),
              loss=categorical_crossentropy_from_logits,
              metrics=[categorical_accuracy_with_variable_timestep])

# model will be used in another following snippet

In [6]:
VGG16_model = Sequential()
VGG16_model.add(BatchNormalization(input_shape=(bottleneck_features_train.shape[1],)))
VGG16_model.add(Dense(embedding_size))
#VGG16_model.add(RepeatVector(3))

# Add Decoder
#
#
#
VGG16_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 512)               2048      
_________________________________________________________________
dense_1 (Dense)              (None, 300)               153900    
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 3, 300)            0         
Total params: 155,948
Trainable params: 154,924
Non-trainable params: 1,024
_________________________________________________________________


In [15]:
from keras.layers import Embedding, Input

### Compile the Model

### Train the Model

In [29]:
from keras.callbacks import ModelCheckpoint

In [ ]:
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.VGG16.flicker8k', 
                               verbose=1, save_best_only=True)

# Model fit

### Load the Model with the Best Validation Loss

In [ ]:
VGG16_model.load_weights('saved_models/weights.best.VGG16.flicker8k')

### Test the Model